In [1]:
import os

In [2]:
os.environ["PATH"]= os.environ["PATH"] + 'C:\\Users\\Tata\\Downloads\\spark-2.2.0-bin-hadoop2.6\\spark-2.2.0-bin-hadoop2.6\\bin'

In [3]:

os.environ["PYSPARK_SUBMIT_ARGS"] = 'pyspark-shell'
os.environ["JAVA_HOME"] = 'C:\\Program Files\\Java\\jdk1.8.0_131'
os.environ["HADOOP_HOME"] = 'C:\\winutils'
#os.environ["PYSPARK_PYTHON"] = 'python3'

os.environ["SCALA_HOME"] = 'C:\\Program Files (x86)\\scala'

In [4]:
os.environ["SPARK_HOME"] = 'C:\\Users\\Tata\\Downloads\\spark-2.2.0-bin-hadoop2.6\\spark-2.2.0-bin-hadoop2.6'

In [5]:
import sys

sys.path.append(os.environ['SPARK_HOME']+"\\python")

In [6]:
sys.path.append(os.environ['SPARK_HOME']+"\\python\\lib\\py4j-0.10.4-src.zip")

In [7]:
import py4j
from pyspark import SparkContext, SparkConf, SQLContext

In [8]:
sc =SparkContext()

In [9]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionModel, LinearRegressionWithSGD
import numpy as np

In [10]:
sqlcontext = SQLContext(sc)

In [11]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionModel, LinearRegressionWithSGD
import numpy as np

In [12]:
from pyspark.ml.classification import GBTClassificationModel

In [13]:
data=[
    LabeledPoint(0.0,[0.0]),
    LabeledPoint(1.0,[1.0]),
    LabeledPoint(3.0,[2.0]),
    LabeledPoint(2.0,[3.0])
]
lrm=LinearRegressionWithSGD.train(sc.parallelize(data),iterations=10,initialWeights=np.array([1.0]))
print(lrm.predict(np.array([1.0])))

C:\Users\Tata\Downloads\spark-2.2.0-bin-hadoop2.6\spark-2.2.0-bin-hadoop2.6\python\pyspark\mllib\regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


0.928638123469


In [14]:
df = sqlcontext.read.format('com.databricks.spark.csv').options(header='true').load('C:\\Users\\Tata\\Downloads\\train.csv')

In [15]:
df

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql import types

def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf = udf(Embarked_transform, types.StringType())
df = df.withColumn('Embarked', my_udf(df['Embarked']))
df.select('Embarked').distinct().collect()

[Row(Embarked='Q'), Row(Embarked='C'), Row(Embarked='S'), Row(Embarked='')]

In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df_t2 = encoder.transform(indexed)


In [18]:
df_t2

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector]

In [19]:
def Get_Title(x):
    y = x.split(", ")[1]
    z = y.split(".")[0]
    return(z)

my_udf2 = udf(Get_Title, types.StringType())
df_t2 = df_t2.withColumn('Title', my_udf2(df_t2['Name']))
df_t2.select('Title').distinct().collect()

[Row(Title='Don'),
 Row(Title='Miss'),
 Row(Title='Col'),
 Row(Title='Rev'),
 Row(Title='Lady'),
 Row(Title='Master'),
 Row(Title='Mme'),
 Row(Title='Capt'),
 Row(Title='Mr'),
 Row(Title='Dr'),
 Row(Title='Mrs'),
 Row(Title='Sir'),
 Row(Title='Jonkheer'),
 Row(Title='Mlle'),
 Row(Title='Major'),
 Row(Title='Ms'),
 Row(Title='the Countess')]

In [20]:
stringIndexer2 = StringIndexer(inputCol="Title", outputCol="TitleIndex")
model2 = stringIndexer2.fit(df_t2)
indexed2 = model2.transform(df_t2)
encoder2 = OneHotEncoder(inputCol="TitleIndex", outputCol="TitleVec")
df_t = encoder2.transform(indexed2)

In [21]:
df_t

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector, Title: string, TitleIndex: double, TitleVec: vector]

In [22]:
def Family(x,y):
    return(str(int(x)+int(y)))

my_udf3 = udf(Family, types.StringType())
df_t = df_t.withColumn('Family', my_udf3(df_t['SibSp'],df_t['Parch']))

In [23]:
df_t

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector, Title: string, TitleIndex: double, TitleVec: vector, Family: string]

In [24]:
from pyspark.sql.functions import col, avg
print(df_t.filter(df_t['Pclass']=="1").agg(avg(col("Age"))).collect()[0][0])
print(df_t.filter(df_t['Pclass']=="2").agg(avg(col("Age"))).collect()[0][0])
print(df_t.filter(df_t['Pclass']=="3").agg(avg(col("Age"))).collect()[0][0])

38.233440860215055
29.87763005780347
25.14061971830986


In [25]:
def parse_age(str_age, Pclass):
    try:
        return float(str_age)
    except:
        if(Pclass=="1"):
            return (38.23)
        if(Pclass=="2"):
            return (29.87)
        if(Pclass=="3"):
            return (25.14)

In [26]:
def IsCabin(str_cabin):
    try:
        return True
    except:
        return False

In [27]:
def IsFamily(str_family):
    if(str_family!="0"):
        return True
    else:
        return False

In [ ]:
def IsFemFirst(str_sex, str_Pclass):
    if(str_sex=="female" and str_Pclass == "1"):
        return True
    else:
        return False

In [ ]:
def IsMaleNotFirst(str_sex, str_Pclass):
    if(str_sex=="male" and (str_Pclass == "2" or str_Pclass=="3")):
        return True
    else:
        return False

In [28]:
def transf(r):
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            int(r.Family), #new
            IsFamily(r.Family), #new
            parse_age(r.Age, r.Pclass), #new
            IsCabin(r.Cabin), #new
        ] + list(r.EmbarkedVec.toArray()) + list(r.TitleVec.toArray()) #new
    )

In [29]:
data = df_t.rdd.map(transf)

In [30]:
data

PythonRDD[105] at RDD at PythonRDD.scala:48

In [31]:
train, test = data.randomSplit([0.7, 0.3])

In [32]:
train.cache()

PythonRDD[106] at RDD at PythonRDD.scala:48

In [33]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
rfc = RandomForest.trainClassifier(train, numClasses=2,
                             categoricalFeaturesInfo={},
                            numTrees=100)

In [39]:
from pyspark.mllib.classification import SVMWithSGD
svm = SVMWithSGD.train(train)

In [45]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
logReg = LogisticRegressionWithLBFGS.train(train)

In [48]:
from pyspark.mllib.classification import NaiveBayes
nb = NaiveBayes.train(train)

In [42]:
def acc(m, test):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    errors = comp.map(lambda x: abs(x[0]-x[1]))
    return 1-errors.sum()/errors.count()

In [66]:
def f1score(m, test):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    precision1 = comp.map(lambda x:  x[0]*x[1])
    precision2 = comp.map(lambda x: x[0]==x[1])
    precision = precision1.sum()/precision2.sum()
    recall1 = comp.map(lambda x:  x[0]*x[1])
    recall2 = comp.map(lambda x: x[1]>0)
    recall = recall1.sum()/recall2.sum()
    f1score = 2*precision*recall/(precision+recall)
    return f1score

In [68]:
print("random forest")
print("accuracy: ", acc(rfc, test))
print("f1 score: ", f1score(rfc,test))
print("SVM")
print("accuracy: ", acc(svm, test))
print("f1 score: ", f1score(svm,test))
print("Logistic Regression")
print("accuracy: ", acc(logReg, test))
print("f1 score: ", f1score(logReg,test))
print("Naive Bayes")
print("accuracy: ", acc(nb, test))
print("f1 score: ", f1score(nb,test))

random forest
accuracy:  0.850187265917603
f1 score:  0.4833836858006043
SVM
accuracy:  0.6217228464419475
f1 score:  0.05185185185185184
Logistic Regression
accuracy:  0.8426966292134832
f1 score:  0.5106382978723404
Naive Bayes
accuracy:  0.722846441948
f1 score:  0.350168350168


In [ ]:
# добавить 5 новых фичей
# 3 фичи высчитываются из имеющихся
# хотя бы одна использует udf

# попробовать 3 новых модели

# f1 меру